In [ ]:
import pandas as pd
df = pd.read_csv('data_horizons.csv', sep=';').dropna(subset=['Bodenart'])
df

In [ ]:
from transformers import AutoImageProcessor, AutoModel

# ChatGPT Input:

```can you write me some python code to finetune a dinov2 checkpoint from hugging face on a computer vision classification task?
```
# ChatGPT Output:

Sure! Here’s a step-by-step guide and Python code for fine-tuning a DINOv2 model (from Hugging Face) for a computer vision classification task using PyTorch and the Hugging Face Transformers library.

## Steps to follow:


* Install the necessary libraries.
* Prepare the dataset for image classification.
* Load a pre-trained DINOv2 model from Hugging Face.
* Prepare the model for fine-tuning.
* Train the model on the dataset.
* Evaluate the performance.

Let’s go through the steps with Python code.

### Step 1: Install Required Libraries
First, ensure that you have the necessary libraries installed. You can install them using pip:

``` pip install transformers datasets torch torchvision evaluate```

### Step 2: Load Dataset

* Step 1: Set Up the DataFrame
Assume that your pandas DataFrame has two columns:

image_path: This contains the full path (or relative path) to each image file.

label: This contains the corresponding label for each image.

* Step 2: Custom Dataset Class
Now, we’ll write a custom PyTorch Dataset that reads from the DataFrame. The Dataset will:

Load the image from the given file path.
Apply transformations (e.g., resize, normalize).
Return the image along with its corresponding label.

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

# Define the image transformations (resize, convert to tensor, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224
    transforms.ToTensor(),          # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

# Custom Dataset class
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): A pandas DataFrame with 'image_path' and 'label' columns.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        """Returns the total number of samples"""
        return len(self.dataframe)

    def __getitem__(self, idx):
        """Retrieves the image and label by index"""
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get the image path and label from the DataFrame
        img_path = self.dataframe.iloc[idx]['image_path']
        label = self.dataframe.iloc[idx]['label']

        # Open the image file
        image = Image.open(img_path).convert("RGB")  # Convert image to RGB if it isn't already

        # Apply the optional transforms (like resize and normalization)
        if self.transform:
            image = self.transform(image)

        return image, label

# Instantiate the dataset
dataset = ImageDataset(df, transform=transform)

# Example: Get the first image and label from the dataset
img, label = dataset[0]
print(f"Image size: {img.size()}, Label: {label}")


### Step 3: DataLoader for Batching
Once we have the Dataset, we can use DataLoader to create batches of images and labels.

In [ ]:
from torch.utils.data import DataLoader

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Iterate through the dataloader (example)
for batch_idx, (images, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx}:")
    print(f"Images shape: {images.shape}, Labels shape: {labels.shape}")
    # Perform your training loop here


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define the image transforms (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 as expected by ViT models
    transforms.ToTensor(),          # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet statistics
])

from transformers import AutoModelForImageClassification

# Load pre-trained DINOv2 model
model_name = "facebook/dinov2-vitb14"
model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=num_classes,  # Use the number of classes from your custom dataset
)

## legacy code

In [ ]:
from PIL import Image
import os
import numpy as np
import cv2
import pandas as pd

###parameters to adapt to the database
nbr_horizons = 15207
nbr_profile = 3101
i_h, i_w = 1500,1500  ##wanted high and width of the pictures and masks

###paths
path_csv = "data_horizons.csv"
path_pictures = "/Users/felix/Library/CloudStorage/Dropbox/data/BGR-Data-Bodenhorizonte/Profilbilder/"
path_images = "images"
path_masks = "masks"


###importation of Excel data
data = pd.read_csv(path_csv, sep=';')
numero = data["PointID"]
lettre = data["Bundesland"]
symboles = data["Horizontsymbol"]
hauteur_inf = data["Untergrenze"]
depthmax = data["max_depth"]
prof_hor = data["Point"]

### extract the main symbol from horizons names

symboles_simples = []
for k in range(nbr_horizons):
    symb = ""
    horizon = symboles[k]
    horizon = str(horizon)
    for car in horizon:
        if car in "AGSBCHMOP":
            symb = car
    if symb == "":
        symboles_simples += "0"
    else:
        symboles_simples += str(symb)


#### creation list of profiles
profile = []
for k in range(nbr_profile):
    if numero[k] < 10:
        profile += [str(str(lettre[k]) + "_000" + str(numero[k]))]
    elif numero[k] < 100:
        profile += [str(str(lettre[k]) + "_00" + str(numero[k]))]
    elif numero[k] < 1000:
        profile += [str(str(lettre[k]) + "_0" + str(numero[k]))]
    else:
        profile += [str(str(lettre[k]) + "_" + str(numero[k]))]



###creation of a list with the real depth of the bottom of the A-horizon

depth_reelle = []
k = 0
for i in range(nbr_profile):
    d = 0
    profil = int(numero[i])
    while prof_hor[k] == profil:
        if symboles_simples[k] == "A" and symboles_simples[k+1] != "A":
            d = hauteur_inf[k]
        k += 1
    if d == 0:
        depth_reelle += [str(profile[i] + "= 00")]
    elif d < 10:
        depth_reelle += [str(profile[i] + "= 0" + str(int(d)))]
    else:
        depth_reelle += [str(profile[i] + "= " + str(int(d)))]

depth_reelle.sort() #sorted by alphabetic order to ease the correlation between the 2 lists

hauteur_image = []
for elt in depth_reelle:
    hauteur_image += [int(elt[-3:])] #get only the depth without the profile name


###list of the depths in the resized pictures
depths = []
for k in range(nbr_profile):
    depth = ((hauteur_image[k] * i_h)/depthmax[k])
    depths.append(depth)


####list images
image_dataset = os.listdir(path_pictures)
images = []
# for profile in image_dataset:
#     path = os.path.join(path_pictures, profile)
#     img = Image.open(path).convert('RGB')
#     images.append(img)


###resize images and convert into array
# y = np.zeros((nbr_profile, i_h, i_w, 1), dtype=np.int32)
# X = np.zeros((nbr_profile, i_h, i_w,3), dtype=np.float32)

# for i in range(nbr_profile) :
#     single_img = images[i]

#     single_img = single_img.resize((i_h, i_w))
#     single_img = np.reshape(single_img, (i_h, i_w, 3))

#     X[i] = single_img


# ### create masks (that has the same dimensions of the image where each pixel is valued at 0 except for the A-horizon where pixel value = 1)

# for k in range (nbr_profile):
#     mask = np.zeros((i_h, i_w, 1), dtype=np.int32)
#     lim = depths[k]

#     if lim == 0:  #if depth =0 : no A-horizon in the profile
#         y[k] = mask
#     else:
#         cv2.rectangle(mask, (0, 0), (i_w, int(lim)), 1, -1)    # assimilate horizon to a rectangle
#         y[k] = mask

# ###save mask and images as png
# i = 1
# j = 1

# for img in X:
#     cv2.imwrite(path_images + "image" + str(i) + ".png", img)
#     i += 1

# for img in y:
#     cv2.imwrite(path_masks + "mask" + str(j) + ".png", img)
#     j += 1
